In [ ]:
%load_ext autoreload
%autoreload

In [2]:
from modeling import process_dataset, produce_idefics_dataset

train_data_path = '/mnt/u14157_ic_nlp_001_files_nfs/nlpdata1/home/ismayilz/cs503-project/data/train/nuscenes/v1_1_train_nus_ext_converted.json'

train_dataset = process_dataset(train_data_path, apply_context="chain")
train_idefics_dataset = produce_idefics_dataset(train_dataset)

100%|██████████| 696/696 [00:00<00:00, 2178.74it/s]


In [1]:
import json
from transformers import AutoProcessor
from modeling import get_objects, parse_object_ref

model_dir = "HuggingFaceM4/idefics2-8b"

processor = AutoProcessor.from_pretrained(
    model_dir,
    do_image_splitting=False
)

# with open("./data/nuscenes/train_idefics_redcircle_vb_chain.json") as f:
#     data = json.load(f)

from modeling import GVQADataCollator

# c = GVQADataCollator(processor)

with open("data/nuscenes/train_idefics_redcircle_vb_chain_od.json") as f:
    train_idefics_dataset = json.load(f)

for sample in train_idefics_dataset:
    question_text = sample["user_message"][0]["content"][-1]["text"]
    question = question_text
    # context = None

    # if "Task:" in question_text:
    #     parts = question_text.split("Task:")
    #     context = parts[0].strip()
    #     question = parts[1].strip()

    question_objects = get_objects(question)
    colors = ["red", "blue", "black", "white", "green", "yellow", "grey", "orange"]
    assert len(question_objects) <= len(colors)
    
    try:
        for object in question_objects:
            parse_object_ref(object)
    except Exception as e:
        print(e)
        print(object)

# _ = c(train_idefics_dataset)

/mnt/u14157_ic_nlp_001_files_nfs/nlpdata1/home/ismayilz/.miniconda3/envs/tff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
text = "Context:\nQ:What are the important objects in the current scene? Those objects will be considered for the future reasoning and driving decision.\nA:There is a brown SUV to the back of the ego vehicle, a black sedan to the back of the ego vehicle, and a green light to the front of the ego vehicle. The IDs of these objects are <c1,CAM_BACK,1088.3,497.5>, <c2,CAM_BACK,864.2,468.3>, and <c3,CAM_FRONT,1043.2,82.2>.\nQ:What is the moving status of object <c1,CAM_BACK,1088.3,497.5>? Please select the correct answer from the following options: A. Stopped. B. Reverse parking. C. Turn left. D. Back up.\nA:C\nQ:What is the object <c1,CAM_BACK,1088.3,497.5>?\nA:Brown SUV\nQ:What is the bounding box and the category of the Brown SUV in back of the ego vehicle?\nA:966.6,403.3,1224.1,591.7 Vehicle\nQ:What is the center coordinate of the Brown SUV in back of the ego vehicle?\nA:1088.3,497.5\nQ:What is the status of the Brown SUV (<c1,CAM_BACK,1088.3,497.5>)?\nA:Moving\nQ:What is the object <c2,CAM_BACK,864.2,468.3>?\nA:Black sedan\nQ:What is the bounding box and the category of the Black sedan in back of the ego vehicle?\nA:816.7,431.6,917.2,505.0 Vehicle\nQ:What is the center coordinate of the Black sedan in back of the ego vehicle?\nA:864.2,468.3\nQ:What is the status of the Black sedan (<c2,CAM_BACK,864.2,468.3>)?\nA:Moving\nQ:What is the object <c3,CAM_FRONT,1043.2,82.2>?\nA:Green light\nQ:What is the bounding box and the category of the Green light in front of the ego vehicle?\nA:676.4,0.0,1452.6,171.5 Traffic element\nQ:What is the center coordinate of the Green light in front of the ego vehicle?\nA:1043.2,82.2\nTask:\nWhat object should the ego vehicle notice first when the ego vehicle is getting to the next possible location? What is the state of the object that is first noticed by the ego vehicle and what action should the ego vehicle take? What object should the ego vehicle notice second when the ego vehicle is getting to the next possible location? What is the state of the object perceived by the ego vehicle as second and what action should the ego vehicle take? What object should the ego vehicle notice third? What is the state of the object perceived by the ego vehicle as third and what action should the ego vehicle take?"
from modeling import get_objects, objects_to_dict

objects = get_objects(text)
dict_o = objects_to_dict(objects)
print(objects)
print(dict_o)

['<c1,CAM_BACK,1088.3,497.5>', '<c2,CAM_BACK,864.2,468.3>', '<c3,CAM_FRONT,1043.2,82.2>', '<c1,CAM_BACK,1088.3,497.5>', '<c1,CAM_BACK,1088.3,497.5>', '<c1,CAM_BACK,1088.3,497.5>', '<c2,CAM_BACK,864.2,468.3>', '<c2,CAM_BACK,864.2,468.3>', '<c3,CAM_FRONT,1043.2,82.2>']
{'CAM_BACK': [[1088.3, 497.5], [864.2, 468.3]], 'CAM_FRONT': [[1043.2, 82.2]]}


In [2]:
from modeling import load_model

model_dir = "HuggingFaceM4/idefics2-8b"
model = load_model(model_dir, eval_mode=False, use_lora=False, use_qlora=True, device="cuda")

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards: 100%|██████████| 7/7 [00:04<00:00,  1.63it/s]


In [3]:
from modeling import GVQADataCollator, load_processor, process_dataset, produce_idefics_dataset
import json, pathlib

MNT_POINT = "/mnt/u14157_ic_nlp_001_files_nfs"

if not pathlib.Path(MNT_POINT).exists():
    MNT_POINT = "/mnt"

train_data_path = f"{MNT_POINT}/nlpdata1/home/ismayilz/cs503-project/data/train/nuscenes/v1_1_train_nus_ext_converted.json"
train_dataset = process_dataset(train_data_path)
train_idefics_dataset = produce_idefics_dataset(train_dataset, apply_context="chain")

# with open("./data/nuscenes/train_idefics_chain.json") as f:
#     train_idefics_dataset = json.load(f)

model_dir = "HuggingFaceM4/idefics2-8b"
processor = load_processor(model_dir)

collator = GVQADataCollator(processor, apply_redcircle=False, apply_input_masking=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
import torch
torch.set_printoptions(profile="full")

batch = collator(train_idefics_dataset[:5])
# print(batch["input_ids"].size())

ValueError: Incorrect image source. Must be a valid URL starting with `http://` or `https://`, a valid path to an image file, or a base64 encoded string. Got /mnt/nlpdata1/home/ismayilz/cs503-project/data/train/nuscenes/samples/CAM_FRONT/n008-2018-09-18-13-10-39-0400__CAM_FRONT__1537291010612404.jpg. Failed with Incorrect padding

In [20]:
context_lengths = []
from tqdm import tqdm

for i in tqdm(range(len(data)//100+1)):
    batch = collator(data[i:i+100])
    context_lengths.append(batch["input_ids"].shape[1])

  0%|          | 1/966 [00:25<6:52:47, 25.67s/it]


KeyboardInterrupt: 